In [1]:
using Laplacians
using PyPlot

objc[36253]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[36253]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[36253]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[36253]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("../../../src/isotonicIPM.jl")
include("../../../src/samplingSolver.jl")

computeCN (generic function with 1 method)

In [3]:
function generateInput(n,i)
    a = wtedChimera(n,i);
    b = zeros(Float64,n);
    b[1] = 1;
    b[n] = -1;
    v = cg(lap(a),b,verbose=false); # use any other solver
    v = v / (v[1] - v[n])

    p = sortperm(v);
    ap = a[p,p];
    # vp = v[p]
    vp = v[p] + rand(n) / sqrt(n);
    at = triu(ap);
    
    return at,vp
end

generateInput (generic function with 1 method)

### First, testing the isotonic regression with augTree solver

In [ ]:
ts_aug = []
accs_aug = []
itercounts_aug = []
epss_aug = []

for i in 1:10
    for eps in [0.01, 0.001, 0.0001]
        a,v = generateInput(100000,i)
        tic()
        (x,acc,itercount) = isotonicIPMrelEps(a,v,eps);
        push!(ts_aug, toc());
        push!(accs_aug, acc)
        push!(itercounts_aug, itercount)
        push!(epss_aug, eps)
    end
    
    println()
    println("Done for ", i)
    println()
end

### Now, isotonic regression with samplingSolver

In [ ]:
ts_samp = []
accs_samp = []
itercounts_samp = []
epss_samp = []

for i in 1:10
    for eps in [0.01, 0.001, 0.0001]
        a,v = generateInput(100000,i)
        tic()
        (x,acc,itercount) = isotonicIPMrelEps(a,v,eps,
            (H -> samplingSolver(adj(H)[1], adj(H)[2], eps=0.5, sampConst=0.3, beta=1e5, tol=1e-1,maxits=1000)));
        push!(ts_samp, toc());
        push!(accs_samp, acc)
        push!(itercounts_samp, itercount)
        push!(epss_samp, eps)
    end
    
    println()
    println("Done for ", i)
    println()
end